# Libraries Used

In [14]:
import vtk
import SimpleITK as sitk
from vtk.util import numpy_support
import numpy as np
import os

# Loading DICOM Files


In [37]:
def load_dicom_series(directory):
    """
    Load a DICOM series from a directory.
    """
    reader = sitk.ImageSeriesReader()
    dicom_files = reader.GetGDCMSeriesFileNames(directory)
    reader.SetFileNames(dicom_files)
    image = reader.Execute()
    return sitk.GetArrayFromImage(image), image.GetSpacing()

# Loading NIfTI Files


In [16]:
def load_nifti_file(file_path):
    """
    Load a NIfTI file.
    """
    image = sitk.ReadImage(file_path)
    return sitk.GetArrayFromImage(image), image.GetSpacing()

# Loading Series Imgs


In [47]:
def load_image_series(directory_or_file):
    """
    Load either a DICOM series or NIfTI file based on input type.
    """
    if os.path.isdir(directory_or_file):  # Directory implies DICOM series
        return load_dicom_series(directory_or_file)
    
    elif directory_or_file.lower().endswith(('*.nii', '*.nii.gz')):  # NIfTI file
        return load_nifti_file(directory_or_file)
    else:
        raise ValueError("Unsupported file type or directory")

# Create dynamic transfer functions


In [24]:
def create_transfer_functions(volume_data):
    """
    Create dynamic color and opacity transfer functions based on the intensity range of the data.
    """
    # Compute the intensity range
    min_intensity = np.min(volume_data)
    max_intensity = np.max(volume_data)
    mid_intensity = (min_intensity + max_intensity) / 2
    # Create the color transfer function
    color_transfer_function = vtk.vtkColorTransferFunction()
    color_transfer_function.AddRGBPoint(min_intensity, 0.0, 0.0, 0.0)  # Black for low-intensity values
    color_transfer_function.AddRGBPoint(mid_intensity, 0.9, 0.6, 0.5)  # Reddish/pink for mid-values
    color_transfer_function.AddRGBPoint(max_intensity, 1.0, 1.0, 1.0)  # White for high-intensity values
    # Create the opacity transfer function
    opacity_transfer_function = vtk.vtkPiecewiseFunction()
    opacity_transfer_function.AddPoint(min_intensity, 0.0)  # Fully transparent for low values
    opacity_transfer_function.AddPoint(mid_intensity, 0.5)  # Semi-transparent for mid-values
    opacity_transfer_function.AddPoint(max_intensity, 1.0)  # Fully opaque for high values

    return color_transfer_function, opacity_transfer_function

# Volume Rendering


In [25]:
def create_volume_renderer(volume_data, spacing):
    """
    Create a volume renderer using VTK for the given 3D volume data.
    """
    # Convert the numpy array to a VTK image
    vtk_image = vtk.vtkImageData()
    depth_array = numpy_support.numpy_to_vtk(num_array=volume_data.ravel(), deep=True, array_type=vtk.VTK_FLOAT)
    vtk_image.SetDimensions(volume_data.shape[::-1])
    vtk_image.GetPointData().SetScalars(depth_array)
    vtk_image.SetSpacing(spacing)
    # Create the volume mapper
    volume_mapper = vtk.vtkSmartVolumeMapper()
    volume_mapper.SetInputData(vtk_image)
    # Set the volume property
    color_transfer_function = vtk.vtkColorTransferFunction()
    opacity_transfer_function = vtk.vtkPiecewiseFunction()
    volume_property = vtk.vtkVolumeProperty()
    volume_property.SetColor(color_transfer_function)
    volume_property.SetScalarOpacity(opacity_transfer_function)
    volume_property.ShadeOn()  # Enable shading for realism
    # Create the volume actor
    volume = vtk.vtkVolume()
    volume.SetMapper(volume_mapper)
    volume.SetProperty(volume_property)
    # Create the renderer and add the volume actor
    renderer = vtk.vtkRenderer()
    renderer.AddVolume(volume)
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark gray background
    renderer.ResetCamera()
    # Set up the render window and interactor
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window.SetSize(800, 600)
    render_interactor = vtk.vtkRenderWindowInteractor()
    render_interactor.SetRenderWindow(render_window)

    return render_window, render_interactor

## Testing DICOM

In [38]:
dicom_directory = r'C:\Study\HIS 2\DICOM-Viewer-Features\series-000001'
print("Loading DICOM series...")
volume_data, spacing = load_image_series(dicom_directory)
print("Setting up the volume renderer...")
render_window, render_interactor = create_volume_renderer(volume_data, spacing)
print("Rendering volume...")
render_window.Render()
render_interactor.Start()

Loading DICOM series...
Setting up the volume renderer...
Rendering volume...


## Testing NIfTI

In [48]:
# dicom_directory = r'C:\Study\HIS 2\DICOM-Viewer-Features\NIFTI'
# print("Loading NIfTI series...")
# volume_data, spacing = load_nifti_file(dicom_directory)
# print("Setting up the volume renderer...")
# render_window, render_interactor = create_volume_renderer(volume_data, spacing)
# print("Rendering volume...")
# render_window.Render()
# render_interactor.Start()
